<a href="https://colab.research.google.com/github/santhimaddipudi/Python-work/blob/master/Python_polars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
df=pd.read_csv("/content/sample_data/Iris.csv")
df_filtered = df[df["SepalLengthCm"] > 5].groupby("Species").agg('sum')  # Filter rows where 'SepalLengthCm' is greater than 5
df_filtered.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
Species,,,,,
Iris-setosa,564,116.9,81.7,33.2,6.1
Iris-versicolor,3562,281.9,131.8,202.9,63.3
Iris-virginica,6168,324.5,146.2,273.1,99.6


In [9]:
import polars as pl

q = (
    pl.scan_csv("/content/sample_data/Iris.csv")
    .filter(pl.col("SepalLengthCm") > 5)
    .group_by("Species")
    .agg(pl.all().sum())
)

df = q.collect()

df

Species,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
str,i64,f64,f64,f64,f64
"""Iris-setosa""",564,116.9,81.7,33.2,6.1
"""Iris-virginica…",6168,324.5,146.2,273.1,99.6
"""Iris-versicolo…",3562,281.9,131.8,202.9,63.3
